In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve
from answer_judge.vqaEval import VQAEval

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/VizWiz/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answer_type,answers,most_likely,responses
0,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,"[Given the question 'What is this can?', is th...",[unanswerable],"[{'answer_confidence': ['yes'], 'answer': ['un...",{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
1,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,"[Given the question 'Are these pants black?', ...",[yes/no],"[{'answer': ['no'], 'answer_confidence': ['yes...",{'embedding': [[[-0.00309753 0.0133667 -0.01...,None
2,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,[Given the question 'What is the brand of this...,[unanswerable],"[{'answer_confidence': ['maybe'], 'answer': ['...",{'embedding': [[[-0.00309753 0.0133667 -0.01...,None
3,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,[Given the question 'What kind of Sam Adam's i...,[other],"[{'answer': ['noble pills'], 'answer_confidenc...",{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None
4,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,[Given the question 'Could you tell me what ki...,[other],"[{'answer': ['unsuitable'], 'answer_confidence...",{'embedding': [[[-4.2114258e-03 -8.1176758e-03...,None


In [5]:
df.shape

(6441, 6)

In [6]:
i=1

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/vizwiz/train/train/VizWiz_train_00002578.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['Unanswerable']

In [9]:
df.iloc[i]['question']

["Given the question 'Are these pants black?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'."]

In [10]:
df.iloc[i]['answer_type']

['yes/no']

In [11]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [12]:
embeddings=np.concatenate(embeddings,axis=0)

In [ ]:
embeddings.shape

(6441, 29, 3584)

In [ ]:
gt_label=df['answer_type'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()
len(gt_label)

6441

In [15]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [16]:
len(rsps)

6441

In [17]:
is_correct=[]
for i,x in enumerate(gt_label):
    if gt_label[i]=='unanswerable' and rsps[i].lower()=='unanswerable':
        is_correct.append(1)
    elif gt_label[i]!='unanswerable' and rsps[i].lower()!='unanswerable':
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [ ]:
is_correct

array([0, 0, 1, ..., 0, 1, 1])

In [19]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 6441 responses. 5258 answer correctly and 1183 answer wrongly


In [20]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 2000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:3152
validset length:2000
testset length:1289
trainset postive: 2551 negative: 601
valset postive: 1642 negative: 358
testset postive: 1065 negative: 224


In [21]:
all_embeddings = embeddings
all_embeddings.shape

(6441, 29, 3584)

In [22]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [23]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel,KernelPCA
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [24]:
M=3584
def evaluator(n_components,i_layer,X,y):
    discriminator=KernelPCA(embedding_train[:,i_layer,:],n_components,M,method='CoP',gamma=1)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return roc_auc_score(y,scores)

In [31]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,11),
    'layer':range(0,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras,all_results=grid_search.search(return_results=True)

Grid Searching for best n_components,layer


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.6710102817792718
found best n_components:1,layer:13 Score: 0.6813260841459182
found best n_components:1,layer:14 Score: 0.6920263474846726
found best n_components:1,layer:16 Score: 0.7031918766458672
found best n_components:1,layer:19 Score: 0.7127676766989433


  0%|          | 0/29 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

In [32]:
np.save('./VizWiz_layers.npy',all_results,allow_pickle=True)

In [40]:
best_paras

{'n_components': 3, 'layer': 20, 'best_score': np.float64(0.7182028661055124)}

In [41]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [43]:
discriminator=PCALinear(embedding_train[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [44]:
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.6623333333333333 AUROC:0.7169490825174762


In [45]:
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.653 AUROC:0.7237870973080318


Train Linear Prob to Get Better Results

In [46]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [47]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [48]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.68 f1:0.778393351800554 auroc:0.701113264011474  b-acc:0.6762466902030009 auc-pr:0.9182839612716984
